In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
import requests
import pandas as pd

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/pramodphilip/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [3]:
url = 'https://quotes.toscrape.com/'
browser.visit(url)

In [4]:
html = browser.html
soup = BeautifulSoup(html,'html.parser')

In [5]:
#quote_list = []

In [6]:
boxes = soup.find_all('div',class_='quote')
#target = boxes[0].a['href']
#browser.links.find_by_href(target).click()

In [7]:
#html = browser.html
#soup = BeautifulSoup(html,'html.parser')

In [8]:
#author_name = soup.find('h3',class_='author-title').text
#author_name

In [9]:
#author_born_date = soup.find('span',class_='author-born-date').text
#author_born_location = soup.find('span',class_='author-born-location').text
#description = soup.find('div',class_='author-description').text.replace('\n', '')

In [10]:
#browser.back()

In [11]:
#quote_text = boxes[0].span.text
#quote_text

In [12]:
#quote_tags = boxes[7].div.text.replace('\n',',').split(',')

#do_not_want = ['','            Tags:','            ']
#quote_tags = [tag for tag in quote_tags if tag not in do_not_want]

#'            ' # boxes[1]
#'            '
#quote_tags.remove('')
#quote_tags.remove('            ')
#quote_tags.remove('            Tags:')
#quote_tags

In [13]:
#quote_mini = {'author_name':author_name,
#              'author_born_date':author_born_date,
#              'author_born_location':author_born_location,
#              'description':description,
#              'quote_text':quote_text,
#              'quote_tags':quote_tags}
#quote_mini

In [14]:
#quote_list.append(quote_mini)
#quote_list

In [15]:
#browser.links.find_by_text('Next ').click()

#html = browser.html
#soup = BeautifulSoup(html,'html.parser')

#boxes = soup.find_all('div',class_='quote')
#boxes

In [16]:
#browser.quit()

In [17]:
quotes_list = []

while len(browser.links.find_by_partial_text('Next ')) > 0:
    for box in boxes:
        
        target = box.a['href']
        browser.links.find_by_href(target).click()
        
        html = browser.html
        soup = BeautifulSoup(html,'html.parser')
        
        quote_mini['author_name'] = soup.find('h3',class_='author-title').text
        
        quote_mini['birth_date'] = soup.find('span',class_='author-born-date').text
        quote_mini['birth_place'] = soup.find('span',class_='author-born-location').text
        quote_mini['description'] = soup.find('div',class_='author-description').text.replace('\n', '')
        
        browser.back()
        
        quote_mini['quote_text'] = box.span.text
        
        quote_tags = box.div.text.replace('\n',',').split(',')
        do_not_want = ['','            Tags:','            ']
        quote_mini['quote_tags'] = [tag for tag in quote_tags if tag not in do_not_want]
        
        #quote_mini = {'author_name':author_name,
        #              'birth_date':author_born_date,
        #              'birth_place':author_born_location,
        #              'description':description,
        #              'quote_text':quote_text,
        #              'quote_tags':quote_tags}
        
        quotes_list.append(quote_mini)
        
    browser.links.find_by_text('Next ').click()

    html = browser.html
    soup = BeautifulSoup(html,'html.parser')

    boxes = soup.find_all('div',class_='quote')

In [18]:
for box in boxes:
    target = box.a['href']
    browser.links.find_by_href(target).click()
        
    html = browser.html
    soup = BeautifulSoup(html,'html.parser')
        
    quote_mini['author_name'] = soup.find('h3',class_='author-title').text
        
    quote_mini['birth_date'] = soup.find('span',class_='author-born-date').text
    quote_mini['birth_place'] = soup.find('span',class_='author-born-location').text
    quote_mini["description"] = soup.find('div',class_='author-description').text.replace('\n', '')
        
    browser.back()
        
    quote_mini['quote_text'] = box.span.text
        
    quote_tags = box.div.text.replace('\n',',').split(',')
    do_not_want = ['','            Tags:','            ']
    quote_mini['quote_tags'] = [tag for tag in quote_tags if tag not in do_not_want]

    #quote_mini = {'author_name':author_name,
    #              'birth_date':author_born_date,
    #              'birth_place':author_born_location,
    #              'description':description,
    #              'quote_text':quote_text,
    #              'quote_tags':quote_tags}
        
    quotes_list.append(quote_mini)

In [19]:
browser.quit()

In [20]:
#quotes_list

In [21]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [22]:
db = client.quotes_db

In [23]:
db.quotes.drop()
db.quotes.insert_many(quotes_list)

In [24]:
list_of_quotes = list(db.quotes.find())
#print(list_of_quotes)

In [25]:
quote_df = pd.DataFrame(list_of_quotes)
quote_df.head()

,_id,author_name,birth_date,birth_place,description,quote_text,quote_tags
0,6053dc193316a74bbd2b83dd,Albert Einstein,"March 14, 1879","in Ulm, Germany","In 1879, Albert Einstein was born in U...",“The world as we have created it is a process ...,"[change, deep-thoughts, thinking, world]"
1,6053dc193316a74bbd2b83de,J.K. Rowling,"July 31, 1965","in Yate, South Gloucestershire, England, The U...",See also: Robert GalbraithAlthough she...,"“It is our choices, Harry, that show what we t...","[abilities, choices]"
2,6053dc193316a74bbd2b83df,Albert Einstein,"March 14, 1879","in Ulm, Germany","In 1879, Albert Einstein was born in U...",“There are only two ways to live your life. On...,"[inspirational, life, live, miracle, miracles]"
3,6053dc193316a74bbd2b83e0,Jane Austen,"December 16, 1775","in Steventon Rectory, Hampshire, The United Ki...",Jane Austen was an English novelist wh...,"“The person, be it gentleman or lady, who has ...","[aliteracy, books, classic, humor]"
4,6053dc193316a74bbd2b83e1,Marilyn Monroe,"June 01, 1926",in The United States,Marilyn Monroe (born Norma Jeane Morte...,"“Imperfection is beauty, madness is genius and...","[be-yourself, inspirational]"


In [26]:
quote_df = quote_df.drop(['_id'],axis=1)

In [27]:
quote_df['quote_id'] = quote_df.index
quote_df.head()

,author_name,birth_date,birth_place,description,quote_text,quote_tags,quote_id
0,Albert Einstein,"March 14, 1879","in Ulm, Germany","In 1879, Albert Einstein was born in U...",“The world as we have created it is a process ...,"[change, deep-thoughts, thinking, world]",0
1,J.K. Rowling,"July 31, 1965","in Yate, South Gloucestershire, England, The U...",See also: Robert GalbraithAlthough she...,"“It is our choices, Harry, that show what we t...","[abilities, choices]",1
2,Albert Einstein,"March 14, 1879","in Ulm, Germany","In 1879, Albert Einstein was born in U...",“There are only two ways to live your life. On...,"[inspirational, life, live, miracle, miracles]",2
3,Jane Austen,"December 16, 1775","in Steventon Rectory, Hampshire, The United Ki...",Jane Austen was an English novelist wh...,"“The person, be it gentleman or lady, who has ...","[aliteracy, books, classic, humor]",3
4,Marilyn Monroe,"June 01, 1926",in The United States,Marilyn Monroe (born Norma Jeane Morte...,"“Imperfection is beauty, madness is genius and...","[be-yourself, inspirational]",4


In [28]:
tags = quote_df['quote_tags']
tags

0              [change, deep-thoughts, thinking, world]
1                                  [abilities, choices]
2        [inspirational, life, live, miracle, miracles]
3                    [aliteracy, books, classic, humor]
4                          [be-yourself, inspirational]
                            ...                        
95                                [better-life-empathy]
96    [books, children, difficult, grown-ups, write,...
97                                              [truth]
98                                      [inspirational]
99                                        [books, mind]
Name: quote_tags, Length: 100, dtype: object

In [40]:
tags = quote_df.explode('quote_tags')
tags = tags[['quote_tags','quote_id']]
tags = tags.set_index('quote_id')
tags.rename(columns = {'quote_tags':'tags'},inplace=True)
tags

,tags
quote_id,
0,change
0,deep-thoughts
0,thinking
0,world
1,abilities
...,...
96,writing
97,truth
98,inspirational


In [ ]:
# To separate the tags in each tag list and make a
# new dataframe in which each tag is listed uniquely and connected to a 
# quote id, do the following (psuedocode):

# Create a "tags" list that will hold mini dictionaries
# E.g. tags = []
# Each mini dictionary - two fields: quote_id and tag

# FOR every row in the dataframe:
#    FOR every tag in the tag list
#       Makes two variables: tag, quote_id
#       Note: quote_id will be based on quote_id value from big loop
#       Note: tag will be based on separated tag from nested loop
#       Mini dictionary - stores the two variables
#       E.g. tags_dict = {quote_id, tag}
#       Append mini dictionary to list
#    END of nested loop
# END of big loop

# PANDAS: make new "tags" dataframe out of "tags" list
# PANDAS: change column names as needed 